In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [3]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import  cross_val_score

import eli5
from eli5.sklearn import  PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [4]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [5]:
ls

data/  day4.ipynb  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [19]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)
df.shape
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X , y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [50]:
df['brand_cat']=df['brand'].map(lambda x: str(x).lower()).factorize()[0]

run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [9]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
def parse_features(x):
  output_dict={}
  if str(x) == 'nan' : return output_dict

  features = literal_eval(x.replace('\\"','"'))

  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [30]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [31]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [32]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df['features_parsed'].map(lambda feats: feats[key] if key in feats else np.nan)

In [33]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_country of origin components:', 'feat_gold karat',
       'feat_looks best on these face shapes:', 'feat_manufacturer number',
       'feat_sole', 'feat_band material', 'feat_colour code',
       'feat_is water-resistant', 'feat_isbn', 'feat_band width'],
      dtype='object', length=525)

In [35]:
df[ df['feat_athlete'].isnull() ].shape[0]

18272

In [36]:
df[ False == df['feat_athlete'].isnull() ].shape[0]

8

In [37]:
df[ False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] * 100

0.0437636761487965

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [44]:
{ k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat']=df['feat_brand'].factorize()[0]
df['feat_color_cat']=df['feat_color'].factorize()[0]
df['feat_gender_cat']=df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat']=df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat']=df['feat_material'].factorize()[0]

df['feat_sport_cat']=df['feat_sport'].factorize()[0]
df['feat_style_cat']=df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [48]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

df[ df.brand == df.feat_brand].shape

(8846, 530)

In [51]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.2743975441708, 4.186308885179781)

In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

feats= ['brand_cat', 'feat_metal type_cat', 'feat_shape_cat','feat_brand_cat','feat_gender_cat','feat_material_cat','feat_sport_cat','feat_style_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [81]:
X = df[feats].values
y= df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

(-57.17387991132396, 4.257786027991963)


Weight,Feature
0.2556 ± 0.0080,brand_cat
0.1033 ± 0.0075,feat_material_cat
0.0255 ± 0.0026,feat_gender_cat
0.0175 ± 0.0015,feat_brand_cat
0.0127 ± 0.0011,feat_shape_cat
0.0093 ± 0.0015,feat_metal type_cat
0.0030 ± 0.0005,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [60]:
df['brand'].value_counts(normalize=True)

nike                      0.097210
puma                      0.033315
ralph lauren              0.028775
vans                      0.021116
new balance               0.020295
                            ...   
smoky                     0.000055
snugpak                   0.000055
owl                       0.000055
principle plastics inc    0.000055
columbia sportswear       0.000055
Name: brand, Length: 1732, dtype: float64

In [70]:
df[df['brand'] =='nike'].features_parsed.sample(5).values

array([{'style': 'air force 1', 'pattern': 'solid', 'material': 'synthetic', 'condition': 'new with box'},
       {},
       {'gender': 'men', 'brand': 'nike', 'fabric content': '100% leather', 'color': 'dark cayenne/university gold'},
       {'style': 'athletic sneakers', 'condition': 'new with box'},
       {'material': 'synthetic', 'gender': 'men', 'size': '9', 'shoe category': "men's shoes", 'color': 'white/anthracite/pure platinum/wolf grey', 'model': 'nikm-flextrainer6w-whiteant455.9m', 'manufacturer part number': 'nikm-flextrainer6w-whiteant455.9m', 'brand': 'nike', 'age group': 'adult'}],
      dtype=object)

In [0]:
!git add day5.ipynb

In [84]:
!git commit -m "Improve "

data/       day5.ipynb         LICENSE      README.md
day4.ipynb  HelloGithub.ipynb  matrix_one/
